# Pushlike interpreter animation

This notebook makes three brief ManimCE animations, each showing a single tree (and all of them are trees) being processed by a little animated interpreter. Everything should run in a Jupyter notebook, as far as I'm aware. Most of the tree-manipulating stuff is called from the `pushTrees.py` module.

**Note** These animations mainly use hand-coded trees and languages. There's a larger `defaultLanguage` defined in `pushTrees.py`, but for various reasons I decided to use a smaller, locally-defined language in the first two of the animations.

## EagerInterpreterAnimation

In the first scene, the interpreter is "eager", in the sense that the functions, when interpreted, produce explicit numerical results. The functions used to calculate those results are encoded in the `explicitCalculationLanguage` structure defined in the scene class. In the event you want to change this, realize it's a relatively fragile process with no strong validation:

- every instruction needs to exist as a key in the language
- the "needs" list is an ordered collection of string names of stacks from which the arguments are pulled
- the "products" list contains items that really want to be objects, with a "stack" target to which each result should be sent, and a "symbolic" value which creates the PushItem result explicitly
- note that the argument order is important for non-commutative functions; check `:sub` and `:div` if you want to understand the trick

## LazyInterpreterAnimation

The same tree is hard-coded, but the language has been changed to allow lazy evaluation. Note that the results of lookup create a new node with a `done` flag set.

## LazyMultitypeInterpreter

Honestly just a big proof of concept. In this case, the language used is the default, which is defined in `pushTrees.py`.



In [8]:
import sys
sys.path.append('..')

from manim import *
from pushTrees import *
from random import shuffle
import copy

In [4]:
%%manim -qm -v WARNING EagerInterpreterAnimation
## this is the ManimCE "magic line", which invokes Manim on the class below when it's invoked; 
# note the `-qm` arg sets medium quality, for faster rendering

class EagerInterpreterAnimation(Scene):
    def construct(self):
        myItems = [[[[":x2", ":k1", ":sub"], ":x1", ":mul"], ":x2", ":sub"], [":x1", ":k2", ":mul"], ":div"]

        explicitCalculationLanguage =  {
            ":x1":{
                "needs":[],
                "products":
                    [{"stack":"number", 
                        "symbolic": lambda:PushItem("19.64", type="number", done = False)}]},
            ":x2":{
                "needs":[],
                "products":
                    [{"stack":"number", 
                        "symbolic": lambda:PushItem("-3.14159", type="number", done = False)}]},
            ":k1":{
                "needs":[],
                "products":
                    [{"stack":"number", 
                        "symbolic": lambda:PushItem(str(3/7), type="number", done = False)}]},
            ":k2":{
                "needs":[],
                "products":
                    [{"stack":"number", 
                        "symbolic": lambda:PushItem(str(2/11), type="number", done = False)}]},
            ":sub":{
                "needs":["number","number"],
                "products":
                    [{"stack":"number",
                        "symbolic": lambda x,y : PushItem("{}".format(float(y.value) - float(x.value)), done = False, type="number")}]},
            ":mul":{
                "needs":["number","number"],
                "products":
                    [{"stack":"number",
                        "symbolic": lambda x,y : PushItem("{}".format(float(y.value) * float(x.value)), done = False, type="number")}]},
            ":div":{
                "needs":["number","number"],
                "products":
                    [{"stack":"number",
                        "symbolic": lambda x,y : PushItem("{}".format(float(y.value) / float(x.value)), done = False, type="number")}]},
            }
        
        ## titles
        explanationTitle = VGroup(
            MarkupText("An Eager Interpreter", font="Didot", slant=ITALIC, font_size=64),
            MarkupText("just working out the answer", font="Didot", slant=NORMAL, font_size=36),
            )
        explanationTitle.arrange(DOWN, buff=0.5)
        self.play(Create(explanationTitle), run_time=2)
        self.wait(5)
        self.play(FadeOut(explanationTitle),run_time=2)
        self.wait()

        ## various settings for interpreter scene
        
        # set this to continue set-aside tokens when :exec is empty, but just once
        retainUnusedTokens = False 
        # constructs the stack Mobjects needed (plus optional "unused" stack at right)
        stacks = createEmptyStacks(["exec", "number"], retainUnusedTokens)
        # screen area used for token processing & function application
        workbench = defaultWorkbench
        # the language defined above, in this case
        language = explicitCalculationLanguage
        
        ## begin animation

        # text Mobject holding the assignments, displayed upper left
        # (the numerical assignments themselves are in explicitCalculationLanguage)
        assignments = Code(code="# assignments\n:x1 = 19.64\n:x2 = -3.14159\n:k1 = 3/7\n:k2 = 2/11", 
                           insert_line_no = False, 
                           background='rectangle', 
                           style = "monokai",
                           language="ruby")
        # show the assignments box
        self.play(Create(assignments.scale(0.75).move_to(5*LEFT+3*UP)))

        # transform the nested array of token strings into a single PushItem 
        theCode = PushItem(myItems)
        # setup
        theCode.mobject.move_to(workbench)
        # draw the stacks
        drawStackStages(self,stacks,retainUnusedTokens)
        # expand program as a subtitle
        listing = showQuickListing(self,theCode,workbench+DOWN,previewScale=3)
        self.wait(2)
        self.play(FadeOut(listing))
        
        # process the PushItem by disassembling (if a codeblock) or looking it up
        handleCodeblockInWorkbench(self,stacks,theCode,workbench) 
        
        # recursively work through every item on `:exec` stack
        processEveryItem(self,stacks,language,workbench,retainUnusedTokens)
        
        self.wait(5)
        
        # grab the top item on the "number" stack
        result = stacks["number"]["contents"][-1]
        # move it back to the workbench region
        self.play(result.mobject.animate.move_to(workbench).scale_to_fit_height(1),run_time=slowTime)
        # label it with the code
        quickCaptionUnderTile(self,result,"final result")
        showQuickListing(self,theCode,workbench+DOWN,previewScale=3)
        
        self.wait(2)
        # we're done
        self.play(*[FadeOut(mob)for mob in self.mobjects])



Manim Community v0.18.1

In [5]:
%%manim -qm -v WARNING LazyInterpreterAnimation
## this is the ManimCE "magic line", which invokes Manim on the class below when it's invoked; 
# note the `-qm` arg sets medium quality, for faster rendering

class LazyInterpreterAnimation(Scene):
    def construct(self):

        # specific tree (flattened postfix) to be rendered 
        myItems = [":x2", ":k1", ":sub", ":x1", ":mul", ":x2", ":sub", ":x1", ":k2", ":mul", ":div"]

        # using an explicit definition of the language here, to show how abstract
        # assignments are handled
        lazyCalculationLanguage =  {
            ":x1":{
                "needs":[],
                "products":
                    [{"stack":"number", 
                        "symbolic": lambda: PushItem(":x1", type="number", done = True)}]},
            ":x2":{
                "needs":[],
                "products":
                    [{"stack":"number", 
                        "symbolic": lambda: PushItem(":x2", type="number", done = True)}]},
            ":k1":{
                "needs":[],
                "products":
                    [{"stack":"number", 
                        "symbolic": lambda: PushItem(":k1", type="number", done = True)}]},
            ":k2":{
                "needs":[],
                "products":
                    [{"stack":"number", 
                        "symbolic": lambda: PushItem(":k2", type="number", done = True)}]},
            ":sub":{
                "needs":["number","number"],
                "products":
                    [{"stack":"number",
                        "symbolic": lambda x,y : PushItem("{} {} :sub".format(y.value, x.value), done = True, type="number")}]},
            ":mul":{
                "needs":["number","number"],
                "products":
                    [{"stack":"number",
                        "symbolic": lambda x,y : PushItem("{} {} :mul".format(y.value, x.value), done = True, type="number")}]},
            ":div":{
                "needs":["number","number"],
                "products":
                    [{"stack":"number",
                        "symbolic": lambda x,y : PushItem("{} {} :div)".format(y.value, x.value), done = True, type="number")}]},
            }
        
        # set this to True to place functions missing arguments onto "unused" stack
        retainUnusedTokens = False 

        # create Mobjects for stack bases
        stacks = createEmptyStacks(["exec", "number"],retainUnusedTokens)

        # set default screen location for handling PushItems
        workbench = defaultWorkbench

        # language defined above
        language = lazyCalculationLanguage

        ## begin animation

        # create assignments Mobject for display
        assignments = Code(code="# assignments\n:x1 : Number\n:x2 : Number\n:k1 : Number\n:k2 : Number", 
                           insert_line_no = False, 
                           background='rectangle', 
                           style = "monokai",
                           language="ruby")

        # show the assignments Mobject, upper left
        self.play(Create(assignments.scale(0.75).move_to(5*LEFT+3*UP)))

        # create a single PushItem with the whole code block in it
        theCode = PushItem(myItems)

        # set it in the workbench region
        theCode.mobject.move_to(workbench)

        # draw the specified stack Mobjects
        drawStackStages(self,stacks,retainUnusedTokens)

        # caption the PushItem
        listing = showQuickListing(self,theCode,workbench+DOWN,previewScale=3)
        self.wait(2)
        self.play(FadeOut(listing))

        # begin processing the PushItem in the workbench region (breaks codeblock into parts)
        handleCodeblockInWorkbench(self,stacks,theCode,workbench)

        # process every item on "exec" stack
        processEveryItem(self,stacks,language,workbench,retainUnusedTokens)
        self.wait(5)
        
        # select top item of "number" stack
        result = stacks["number"]["contents"][-1]
        
        # move it to the workbench
        self.play(result.mobject.animate.move_to(workbench).scale_to_fit_height(1),run_time=slowTime)
        quickCaptionUnderTile(self,result,"final result")
        showQuickListing(self,theCode,workbench+DOWN,previewScale=3)
        
        # we're done
        self.wait(2)
        self.play(*[FadeOut(mob)for mob in self.mobjects])



Manim Community v0.18.1

In [6]:
%%manim -qm -v WARNING LazyMultitypeInterpreter
## this is the ManimCE "magic line", which invokes Manim on the class below when it's invoked; 
# note the `-qm` arg sets medium quality, for faster rendering


class LazyMultitypeInterpreter(MovingCameraScene):
    def construct(self):

        # slightly more complicated postfix tree
        myItems = [ [ [ [ ":x2", [ ":s1", ":len" ], ":sub" ], ":x1", ":mul"], [ ":p1", ":p2", ":dist" ], ":sub" ], [ ":b1", ":x1", [ [ ":p1", ":x-of" ], [ ":p2", ":y-of" ], ":k1", ":x2", ":iflte" ], ":ifelse" ], ":pdiv" ]

        # there aren't going to be any missing arguments; it's a tree!
        retainUnusedTokens = False 

        # you do have to specify all stacks here, explicitly
        stacks = createEmptyStacks(["exec", "number", "string", "bool", "point"],retainUnusedTokens)

        # set default screen location for handling PushItems
        workbench = defaultWorkbench

        # this time we're using the language defined in the 'pushTrees.py' module
        language = defaultLanguage

        ## begin animation 

        # construct assignments Mobject
        assignments = Code(code="# assignments\n:x1: Number\t:x2: Number\n:k1: Number\t:k2: Number\n:s1: String\n:b1: Boolean\n:p1: 2dPoint\t:p2: 2dPoint", 
                           insert_line_no = False, 
                           background='rectangle', 
                           style = "monokai",
                           language="text")
        # display it
        self.play(Create(assignments.scale(0.75).move_to(4.5*LEFT+3*UP).fade()))

        # create the initial PushItem holding the entire tree
        theCode = PushItem(myItems)
        # prep for processing
        theCode.mobject.move_to(workbench)
        # draw the stack bases & labels
        drawStackStages(self,stacks,retainUnusedTokens)
        # show the listing
        listing = showQuickListing(self,theCode,workbench+DOWN,previewScale=3)
        self.wait(2)
        self.play(FadeOut(listing))

        # break up the initial codeblock PushItem (currently in workbench)
        handleCodeblockInWorkbench(self,stacks,theCode,workbench) 
        # recursively handle everything on "exec" stack
        processEveryItem(self,stacks,language,workbench,retainUnusedTokens)
        self.wait(5)
        
        # grab top item on "number stack
        result = stacks["number"]["contents"][-1]
        # move it to the workbench
        self.play(result.mobject.animate.move_to(workbench).scale_to_fit_height(1),run_time=slowTime)
        quickCaptionUnderTile(self,result,"final result")
        showQuickListing(self,theCode,workbench+DOWN,previewScale=3)

        # prep for zooming
        self.camera.frame.save_state()
        # zoom in on the result item
        self.play(self.camera.frame.animate.move_to(result.mobject.get_center()).scale(0.33))
        self.wait(2)
        self.play(Restore(self.camera.frame))
        self.wait(2)
        self.play(*[FadeOut(mob)for mob in self.mobjects])



Manim Community v0.18.1